# SQL

In [1]:
import pandas as pd 
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [2]:
students = pd.read_csv("students.csv")
password = '#Titpfmss109'
db_name = 'project2'

In [3]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)

In [4]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [5]:
students.to_sql(name="students", con=connection, if_exists='replace', index=False)

30

## SQL Tables

In [6]:
query1 = "SELECT * FROM students"
studs = pd.read_sql(query1, con=connection)

In [7]:
courses = pd.read_csv("courses.csv")

In [8]:
courses.to_sql(name='courses', con=connection, if_exists='replace', index=False)
cors = pd.read_sql("SELECT * FROM courses", con=connection)

In [9]:
grades = pd.read_csv("grades.csv")

In [10]:
grades.to_sql(name='grades', con=connection, if_exists='replace', index=False)
grad = pd.read_sql("SELECT * FROM grades", con=connection)
grad

,grade_id,student_id,assignment_id,grade,course_id
0,0,0,4,85,1
1,1,0,5,70,1
2,2,0,6,90,1
3,3,1,7,70,2
4,4,1,8,80,2
...,...,...,...,...,...
102,102,28,9,79,2
103,103,28,10,82,2
104,104,29,26,80,7
105,105,29,27,100,7


In [11]:
assignments = pd.read_csv("assignments.csv")

In [12]:
assignments.to_sql(name='assignments', con=connection, if_exists='replace', index=False)
assign = pd.read_sql("SELECT * FROM assignments", con=connection)

### Table with all columns (no aggregations)

In [13]:
full_query = """SELECT g.grade_id, s.student_id, s.name, a.assignment_id, a.assignment_name, g.grade, a.weight, a.due_date, c.course_id, c.course_name, s.gpa, s.major, s.year, s.gender, s.birthday, s.email, c.instructor, c.day, c.time, c.grade_avg, c.credit_hrs, c.enrollment, c.syllabi, c.description
FROM courses AS c
JOIN assignments AS a
ON c.course_id = a.course_id
JOIN grades AS g
ON g.assignment_id = a.assignment_id
JOIN students AS s
ON g.student_id = s.student_id
ORDER by g.grade_id;"""
full_cols = pd.read_sql(full_query, con=connection)
full_cols.to_sql(name='full_cols', con=connection, if_exists='replace', index=False)
# talk about full data drawbacks
full_cols

,grade_id,student_id,name,assignment_id,assignment_name,grade,weight,due_date,course_id,course_name,...,birthday,email,instructor,day,time,grade_avg,credit_hrs,enrollment,syllabi,description
0,0,0,Ethan Patel,4,Research Paper,85,0.40,4/9/23,1,"""Exploring Contemporary Art Movements""",...,12/05/2003,ethan.patel@college.edu,John Adams,T TH,1:00 PM,A,3,18,1,This course explores various art movements of ...
1,1,0,Ethan Patel,5,Midterm Exam,70,0.25,2/25/23,1,"""Exploring Contemporary Art Movements""",...,12/05/2003,ethan.patel@college.edu,John Adams,T TH,1:00 PM,A,3,18,1,This course explores various art movements of ...
2,2,0,Ethan Patel,6,Final Exam,90,0.35,5/17/23,1,"""Exploring Contemporary Art Movements""",...,12/05/2003,ethan.patel@college.edu,John Adams,T TH,1:00 PM,A,3,18,1,This course explores various art movements of ...
3,3,1,Mia Johnson,7,Exam 1,70,0.25,3/3/23,2,"""Introduction to Microeconomics: Principles an...",...,02/14/2002,mia.johnson@college.edu,Cynthia Turner,T W TH,1:00 PM,C,4,25,0,This course introduces the principles of micro...
4,4,1,Mia Johnson,8,Problem Set,80,0.15,2/24/23,2,"""Introduction to Microeconomics: Principles an...",...,02/14/2002,mia.johnson@college.edu,Cynthia Turner,T W TH,1:00 PM,C,4,25,0,This course introduces the principles of micro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,102,28,Grace Carter,9,Exam 2,79,0.20,4/13/23,2,"""Introduction to Microeconomics: Principles an...",...,01/07/2000,grace.carter@college.edu,Cynthia Turner,T W TH,1:00 PM,C,4,25,0,This course introduces the principles of micro...
103,103,28,Grace Carter,10,Final Exam,82,0.40,5/12/23,2,"""Introduction to Microeconomics: Principles an...",...,01/07/2000,grace.carter@college.edu,Cynthia Turner,T W TH,1:00 PM,C,4,25,0,This course introduces the principles of micro...
104,104,29,Elijah Bennett,26,Deliverable,80,0.20,3/2/23,7,"""Introduction to Digital Marketing: Strategies...",...,06/13/2004,elijah.bennett@college.edu,Mark Bennett,M W,3:00 PM,A,3,14,1,This course provides an overview of digital ma...
105,105,29,Elijah Bennett,27,Project 1,100,0.30,4/6/23,7,"""Introduction to Digital Marketing: Strategies...",...,06/13/2004,elijah.bennett@college.edu,Mark Bennett,M W,3:00 PM,A,3,14,1,This course provides an overview of digital ma...


### Different ways of calculating grade from performances

In [14]:
final_grade_query = """WITH final_grades AS (SELECT student_id, SUM(assignments.weight * grades.grade) AS result FROM assignments JOIN grades ON assignments.assignment_id = grades.assignment_id GROUP BY student_id)
SELECT s.student_id, s.name, s.gpa, s.major, s.year, s.gender, s.birthday, s.email, c.course_name, c.instructor, c.grade_avg, c.day, c.time, c.credit_hrs, fg.result AS final_grade
FROM final_grades AS fg
JOIN students AS s
ON s.student_id = fg.student_id
JOIN courses AS c
ON s.course_id = c.course_id;"""

final_grade_full = pd.read_sql(final_grade_query, con=connection)

final_grade_sub = pd.read_sql("SELECT student_id, SUM(assignments.weight * grades.grade) AS final_grade FROM assignments JOIN grades ON assignments.assignment_id = grades.assignment_id GROUP BY student_id", con=connection)
final_grade_full

,student_id,name,gpa,major,year,gender,birthday,email,course_name,instructor,grade_avg,day,time,credit_hrs,final_grade
0,0,Ethan Patel,3.2,Psychology,3,male,12/05/2003,ethan.patel@college.edu,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3,83.00
1,1,Mia Johnson,2.8,Computer Science,3,female,02/14/2002,mia.johnson@college.edu,"""Introduction to Microeconomics: Principles an...",Cynthia Turner,C,T W TH,1:00 PM,4,79.10
2,2,Noah Thompson,2.9,Business Administration,1,male,06/28/2005,noah.thompson@college.edu,"""Introduction to Psychology: Understanding the...",James Wright,B,M,2:00 PM,2,71.50
3,3,Ava Davis,3.7,Nursing,4,female,03/10/2001,ava.davis@college.edu,"""Introduction to Psychology: Understanding the...",James Wright,B,M,2:00 PM,2,86.50
4,4,Oliver Smith,2.5,Biology,2,nonbinary,07/19/2004,oliver.smith@college.edu,"""Introduction to Psychology: Understanding the...",James Wright,B,M,2:00 PM,2,80.00
5,5,Sophia Rodriguez,3.9,Communication Studies,5,female,09/22/2000,sophia.rodriguez@college.edu,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3,90.55
6,6,Lucas Taylor,3.1,English Literature,3,male,10/08/2003,lucas.taylor@college.edu,"""Introduction to Digital Marketing: Strategies...",Mark Bennett,A,M W,3:00 PM,3,91.50
7,7,Isabella Wilson,4.0,Mechanical Engineering,3,female,05/11/2002,isabella.wilson@college.edu,"""Introduction to Environmental Studies: Sustai...",Pamela Cooper,B,M TH F,2:00 PM,4,95.00
8,8,Liam Nguyen,2.1,Marketing,1,male,11/13/2005,liam.nguyen@college.edu,"""Introduction to Environmental Studies: Sustai...",Pamela Cooper,B,M TH F,2:00 PM,4,72.50
9,9,Olivia Campbell,3.8,Economics,1,female,01/16/2004,olivia.campbell@college.edu,"""Fundamentals of Biochemistry and Molecular Bi...",Robert Phillips,C,M T TH F,9:00 AM,5,90.60


In [15]:
stud_grade = pd.merge(students, final_grade_sub[['student_id','final_grade']], on='student_id', how='inner')
stud_grade

,student_id,name,gpa,major,year,gender,birthday,email,course_id,final_grade
0,0,Ethan Patel,3.2,Psychology,3,male,12/05/2003,ethan.patel@college.edu,1,83.00
1,1,Mia Johnson,2.8,Computer Science,3,female,02/14/2002,mia.johnson@college.edu,2,79.10
2,2,Noah Thompson,2.9,Business Administration,1,male,06/28/2005,noah.thompson@college.edu,3,71.50
3,3,Ava Davis,3.7,Nursing,4,female,03/10/2001,ava.davis@college.edu,3,86.50
4,4,Oliver Smith,2.5,Biology,2,nonbinary,07/19/2004,oliver.smith@college.edu,3,80.00
5,5,Sophia Rodriguez,3.9,Communication Studies,5,female,09/22/2000,sophia.rodriguez@college.edu,1,90.55
6,6,Lucas Taylor,3.1,English Literature,3,male,10/08/2003,lucas.taylor@college.edu,7,91.50
7,7,Isabella Wilson,4.0,Mechanical Engineering,3,female,05/11/2002,isabella.wilson@college.edu,6,95.00
8,8,Liam Nguyen,2.1,Marketing,1,male,11/13/2005,liam.nguyen@college.edu,6,72.50
9,9,Olivia Campbell,3.8,Economics,1,female,01/16/2004,olivia.campbell@college.edu,5,90.60


In [16]:
full_df = pd.merge(stud_grade,courses[['course_id','course_name','instructor','grade_avg','day','time','credit_hrs']],on='course_id', how='inner')
full_df

,student_id,name,gpa,major,year,gender,birthday,email,course_id,final_grade,course_name,instructor,grade_avg,day,time,credit_hrs
0,0,Ethan Patel,3.2,Psychology,3,male,12/05/2003,ethan.patel@college.edu,1,83.00,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
1,5,Sophia Rodriguez,3.9,Communication Studies,5,female,09/22/2000,sophia.rodriguez@college.edu,1,90.55,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
2,12,Gabriel Robinson,3.1,Graphic Design,1,female,07/02/2005,gabriel.robinson@college.edu,1,83.35,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
3,17,Harper Garcia,2.4,Finance,5,female,02/23/2000,harper.garcia@college.edu,1,68.75,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
4,22,Lily Mitchell,2.2,Music Performance,4,female,09/29/2000,lily.mitchell@college.edu,1,84.50,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
5,23,Samuel Hernandez,2.7,Journalism,1,male,12/09/2004,samuel.hernandez@college.edu,1,83.50,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
6,1,Mia Johnson,2.8,Computer Science,3,female,02/14/2002,mia.johnson@college.edu,2,79.10,"""Introduction to Microeconomics: Principles an...",Cynthia Turner,C,T W TH,1:00 PM,4
7,11,Emma Wright,3.8,Sociology,3,female,04/25/2003,emma.wright@college.edu,2,84.20,"""Introduction to Microeconomics: Principles an...",Cynthia Turner,C,T W TH,1:00 PM,4
8,16,Elijah Lewis,3.5,Chemistry,1,male,11/20/2004,elijah.lewis@college.edu,2,84.25,"""Introduction to Microeconomics: Principles an...",Cynthia Turner,C,T W TH,1:00 PM,4
9,21,Harper Robinson,3.1,Psychology,2,male,07/16/2003,harper.robinson@college.edu,2,83.65,"""Introduction to Microeconomics: Principles an...",Cynthia Turner,C,T W TH,1:00 PM,4


In [17]:
full_df.to_sql(name="students_information", con=connection, if_exists='replace', index=False)

30

# MONGO

We created a few different collections for different purposes and messing around with dictionaries -- the one that we thought was displayed the nicest and also worked best with aggregations is titled `final`.

In [18]:
mongo_students = full_df.to_dict('records')
#for i in mongo_students: 
   # print(i["course_id"])
students_dict = studs.to_dict('records')
courses_dict = cors.to_dict('index')
for i in courses_dict.values(): 
    for i in students_dict: 
        i['course_id'] = courses_dict[i['course_id']]

TypeError: unhashable type: 'dict'

In [21]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import pprint

In [22]:
client = MongoClient('localhost', 27017)

In [23]:
db = client.project2

In [24]:
collection1 = db.unnested

In [25]:
collection1.insert_many(mongo_students)

In [26]:
collection2 = db.nested_courses

In [27]:
collection2.insert_many(students_dict)

In [28]:
# UNNESTED
stud_grade = pd.merge(students, final_grade_sub[['student_id','final_grade']],on='student_id', how='inner')
grades_of_students= pd.merge(assign,grad[['assignment_id','student_id','grade']], on ='assignment_id', how='outer')
dic = grades_of_students.to_dict('records')
dics ={}
for i in dic: 
    if i['student_id'] not in dics: 
        dics[i['student_id']] = [] 
        dics[i['student_id']].append((i['assignment_name'],i['grade'], i['weight']))
    else: 
        dics[i['student_id']].append((i['assignment_name'],i['grade'], i['weight']))

In [29]:
# NESTED FULL
stud_grade = pd.merge(students, final_grade_sub[['student_id','final_grade']],on='student_id', how='inner')
grades_of_students= pd.merge(assign,grad[['assignment_id','student_id','grade']], on ='assignment_id', how='outer')
dic = grades_of_students.to_dict('records')
dics ={}
for elem in dic: 
    if elem['student_id'] not in dics: 
        dics[elem['student_id']] = {elem["assignment_name"]: {"grade": elem["grade"], "weight": elem["weight"]}}
    else: 
        dics[elem['student_id']][elem['assignment_name']] = {"grade": elem['grade'], "weight": elem['weight']}

In [30]:
# FINAL
stud_grade = pd.merge(students, final_grade_sub[['student_id','final_grade']],on='student_id', how='inner')
grades_of_students= pd.merge(assign,grad[['assignment_id','student_id','grade']], on ='assignment_id', how='outer')
dic = grades_of_students.to_dict('records')
dics ={}
for elem in dic: 
    if elem['student_id'] not in dics: 
        dics[elem['student_id']] = [{"grade": elem["grade"], "weight": elem["weight"], "name": elem["assignment_name"]}]
    else: 
        dics[elem['student_id']].append({"grade": elem['grade'], "weight": elem['weight'], "name": elem["assignment_name"]})

In [31]:
for i in dics:
    print(i)

24
25
0
5
12
17
22
23
1
11
16
21
28
2
3
4
15
20
26
13
27
9
10
18
7
8
19
6
14
29


In [32]:
studes = studs.to_dict('records')
cours = cors.to_dict('records')
cors
for i in studes: 
    i['course_id'] = cours[i['course_id']]
lis = []
count = 0
for i in studes: 
    i['grades']= dics[i['student_id']]

In [33]:
collection3 = db.nested_full

In [34]:
collection3.insert_many(studes)

In [35]:
collection4 = db.nested_numerical

In [36]:
collection4.insert_many(studes)

In [37]:
collection5 = db.final

In [38]:
collection5.insert_many(studes)

### Aggregations on `final` collection

Note: Visualized and completed all this aggregation in MongoDB Compass, as it was a lot easier to look at the results.

In [45]:
# GRADE CALCULATION
collection5.aggregate([
    {
        '$unwind': {
            'path': '$grades'
        }
    }, {
        '$project': {
            'student_id': 1, 
            'name': 1, 
            'grades': 1
        }
    }, {
        '$group': {
            '_id': '$name', 
            'final_grade': {
                '$sum': {
                    '$multiply': [
                        '$grades.grade', '$grades.weight'
                    ]
                }
            }
        }
    }
])

In [46]:
# AVG GPA BY YEAR
collection5.aggregate([
    {
        '$group': {
            '_id': '$year', 
            'avg_gpa': {
                '$avg': '$gpa'
            }
        }
    }
])

In [47]:
# AVG GPA BY MAJOR
collection5.aggregate([
    {
        '$group': {
            '_id': '$major', 
            'avg_gpa': {
                '$avg': '$gpa'
            }
        }
    }
])

In [48]:
# AVG GPA BY COURSE
collection5.aggregate([
    {
        '$group': {
            '_id': '$course_id.course_name', 
            'avg_gpa': {
                '$avg': '$gpa'
            }
        }
    }
])

# NEO4J

In [39]:
from py2neo import Graph, Node, Relationship

In [41]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [42]:
stud_grade

,student_id,name,gpa,major,year,gender,birthday,email,course_id,final_grade
0,0,Ethan Patel,3.2,Psychology,3,male,12/05/2003,ethan.patel@college.edu,1,83.00
1,1,Mia Johnson,2.8,Computer Science,3,female,02/14/2002,mia.johnson@college.edu,2,79.10
2,2,Noah Thompson,2.9,Business Administration,1,male,06/28/2005,noah.thompson@college.edu,3,71.50
3,3,Ava Davis,3.7,Nursing,4,female,03/10/2001,ava.davis@college.edu,3,86.50
4,4,Oliver Smith,2.5,Biology,2,nonbinary,07/19/2004,oliver.smith@college.edu,3,80.00
5,5,Sophia Rodriguez,3.9,Communication Studies,5,female,09/22/2000,sophia.rodriguez@college.edu,1,90.55
6,6,Lucas Taylor,3.1,English Literature,3,male,10/08/2003,lucas.taylor@college.edu,7,91.50
7,7,Isabella Wilson,4.0,Mechanical Engineering,3,female,05/11/2002,isabella.wilson@college.edu,6,95.00
8,8,Liam Nguyen,2.1,Marketing,1,male,11/13/2005,liam.nguyen@college.edu,6,72.50
9,9,Olivia Campbell,3.8,Economics,1,female,01/16/2004,olivia.campbell@college.edu,5,90.60


In [48]:
full_df.head()

,student_id,name,gpa,major,year,gender,birthday,email,course_id,final_grade,course_name,instructor,grade_avg,day,time,credit_hrs
0,0,Ethan Patel,3.2,Psychology,3,male,12/05/2003,ethan.patel@college.edu,1,83.00,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
1,5,Sophia Rodriguez,3.9,Communication Studies,5,female,09/22/2000,sophia.rodriguez@college.edu,1,90.55,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
2,12,Gabriel Robinson,3.1,Graphic Design,1,female,07/02/2005,gabriel.robinson@college.edu,1,83.35,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
3,17,Harper Garcia,2.4,Finance,5,female,02/23/2000,harper.garcia@college.edu,1,68.75,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3
4,22,Lily Mitchell,2.2,Music Performance,4,female,09/29/2000,lily.mitchell@college.edu,1,84.50,"""Exploring Contemporary Art Movements""",John Adams,A,T TH,1:00 PM,3


In [46]:
#for i in range(len(stud_grade)):
    #row = stud_grade.iloc[i]
   # graph.run("CREATE (:STUDENT {name:'" + row['name'] + "',gpa:" + str(row['gpa']) + ",major:'" + row['major']+"'})")

In [52]:
student_ids = full_df['student_id']
student_name = full_df['name']
student_gpa = full_df['gpa']
student_major = full_df['major']
student_year = full_df['year']
student_gender = full_df['gender']
student_birthday = full_df['birthday']
student_email = full_df['email']

course_ids = courses["course_id"]
course_names = courses["course_name"]
instructors = courses["instructor"]
grade_avgs = courses["grade_avg"]
days = courses["day"]
times = courses["time"]
credit_hrss = courses["credit_hrs"]

students = pd.DataFrame({"student_id": student_ids, "name": student_name, "gpa": student_gpa, "major": student_major, "year": student_year, "gender": student_gender, "birthday": student_birthday, "email": student_email})
courses = pd.DataFrame({"course_id": course_ids, "course_name": course_names, "instructor": instructors, "grade_avg": grade_avgs, "day": days, "time": times, "credit_hrs": credit_hrss})

In [53]:
#students
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("CREATE (:STUDENT {student_id: " + str(row.student_id) + ", name: '" + row['name'] + "', gpa: " + str(row.gpa) + ", major: '" + row['major'] + "', year: " + str(row.year) + ", gender: '" + row['gender'] + "', birthday: '" + row['birthday'] + "', email: '" + row['email'] + "'})")

In [54]:
#courses
for i in range(len(courses)):
    row = courses.iloc[i]
    graph.run("CREATE (:COURSES {course_name:'" + row['course_name'] + "',grade_avg:'" + row.grade_avg +  "',instructor:'" + row.instructor + "'})")

In [55]:
for i in range(len(full_df)):
    row = full_df.iloc[i]
    graph.run(f""" MATCH
    (p:PERSON), 
    (c:COURSES)
    WHERE p.name = "{row['name']}" AND c.course_name ={row['course_name']} 
    CREATE (p)-[r:TAKES]->(c)""")

In [ ]:
#assignments



In [51]:
graph.run("MATCH (n) DETACH DELETE n")


(No data)